In [3]:
# get_ipython().magic('matplotlib notebook')
get_ipython().magic('matplotlib inline')
get_ipython().magic('load_ext autoreload')
get_ipython().magic('autoreload 2')
#_______________________________________________________________________________________________
import yaml
import pkg_resources
from jinja2 import Environment, FileSystemLoader
import json
import sys
import os
from tripyview import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
#_________________________________________________________________________________________________
# load yaml setting file
yaml_filename='diagrun_test.yml'
# yaml_filename='diagrun_tke+idemix_tuning.yml'

with open(yaml_filename) as file:
    yaml_settings = yaml.load(file, Loader=yaml.FullLoader)

#_________________________________________________________________________________________________    
# name of workflow runs --> also folder name 
workflow_name = yaml_settings['workflow_name']

# setup data input paths & input names
input_paths = yaml_settings['input_paths']
if 'input_names' in yaml_settings:
    input_names = yaml_settings['input_names']
else:
    input_names = list()
    for path in input_paths: 
        input_names.append(path.split('/')[-3])
if len(input_names) != len(input_paths): raise ValueError("The size of input_names & input_paths is not equal") 

# setup save path    
if 'save_path' in yaml_settings: 
    save_path = f"{yaml_settings['save_path']}/{workflow_name}"
else:
    save_path = f"./results/{workflow_name}" 
save_path = os.path.expanduser(save_path)
save_path = os.path.abspath(save_path)

    
#_________________________________________________________________________________________________    
# actualize settings dictionary    
yaml_settings['input_paths']       = input_paths
yaml_settings['input_names']       = input_names
yaml_settings['workflow_name']     = workflow_name
yaml_settings['workflow_settings'] = None
yaml_settings['save_path']         = save_path
yaml_settings['save_path_nb' ]     = os.path.join(save_path, "notebooks")
yaml_settings['save_path_fig']     = os.path.join(save_path, "figures")    

#_________________________________________________________________________________________________
# create save directory if they do not exist
if not os.path.exists(yaml_settings['save_path']):
    print(f' --> mkdir: {yaml_settings["save_path"]}')
    os.makedirs(yaml_settings["save_path"])
    print(f' --> mkdir: {yaml_settings["save_path_nb"]}')
    os.makedirs(yaml_settings["save_path_nb"])
    print(f' --> mkdir: {yaml_settings["save_path_fig"]}')
    os.makedirs(yaml_settings["save_path_fig"])
    
#_________________________________________________________________________________________________    
# initialise/create webpage interface .json file
fname_json = f"{yaml_settings['workflow_name']}.json"
save_path_json = os.path.join(yaml_settings['save_path'], fname_json)
if os.path.exists(save_path_json):
    with open(save_path_json) as json_file:
        webpages = json.load(json_file)
        print(f"Data on previous runs exist in {save_path_json}, \n")
        print("they will be used to generate output for diagnostics you do not run this time.")
else:
    webpages = {}
    webpages["analyses"] = {}

webpages["general"] = {}
webpages["general"]["name"] = yaml_settings["workflow_name"]

#_________________________________________________________________________________________________
# define analyses drivers
analyses_opt = {}
analyses_opt["hslice"         ] = drive_hslice
analyses_opt["hslice_np"      ] = drive_hslice
analyses_opt["hslice_sp"      ] = drive_hslice
analyses_opt["hslice_clim"    ] = drive_hslice_clim
analyses_opt["hovm"           ] = drive_hovm
analyses_opt["hovm_clim"      ] = drive_hovm_clim
analyses_opt["xmoc"           ] = drive_xmoc
analyses_opt["vprofile"       ] = drive_vprofile
analyses_opt["vprofile_clim"  ] = drive_vprofile_clim
analyses_opt["transect"       ] = drive_transect
analyses_opt["transect_clim"  ] = drive_transect_clim
analyses_opt["zmeantrans"     ] = drive_zmeantrans
analyses_opt["zmeantrans_clim"] = drive_zmeantrans_clim

Data on previous runs exist in /home/ollie/pscholz/tripyview_github/Results/TRR181_TKE+IDEMIX_test/TRR181_TKE+IDEMIX_test.json, 

they will be used to generate output for diagnostics you do not run this time.


In [ ]:
#_________________________________________________________________________________________________
# loop over all analyses
for analysis in analyses_opt:
    # check if analysis is in input yaml settings
    if analysis in yaml_settings:
        print(f" --> compute {analysis}:")
        # drive specific analysis
        webpage = analyses_opt[analysis](yaml_settings, analysis)
        webpages["analyses"][analysis] = webpage
        
        # write linked analysis to .json file
        with open(save_path_json, "w") as fp: json.dump(webpages, fp)
            

Black is not installed, parameters wont be formatted


 --> compute hslice:
 --> drive_hslice: hslice


Executing:   0%|          | 0/6 [00:00<?, ?cell/s]

Black is not installed, parameters wont be formatted


Executing:   0%|          | 0/6 [00:00<?, ?cell/s]

In [ ]:
#_________________________________________________________________________________________________            
# save everything to .html
fname_html     = f"{yaml_settings['workflow_name']}.html"
save_path_html = os.path.join(yaml_settings['save_path'], fname_html)
ofile          = open(save_path_html, "w")
template       = env.get_template("experiment.html")
output         = template.render(webpages)
ofile.write(output)
ofile.close()       

In [ ]:
#_________________________________________________________________________________________________            
# render page
# render_main_page()
